In [84]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import plotly.graph_objects as go

from scipy.signal import savgol_filter
from scipy.fft import fft, ifft
from scipy.signal import butter,filtfilt

# %matplotlib widget

In [85]:
def modifyTimestamps(df):

    df = df.dropna()
    df = df.drop(df.filter(regex='Name..').columns, axis=1)
    df = df.rename(columns={"Name": "Time"})

    return df


def LDAP_to_time(t_array):

    timestamp_array = []

    for timestamp in t_array:

        if type(timestamp) == pd._libs.tslibs.timestamps.Timestamp:
            timestamp_array.append(timestamp)

        else:
            value = datetime.datetime(1601, 1, 1) + datetime.timedelta(seconds=timestamp/10000000)
            timestamp_array.append(value)

    return np.asarray(timestamp_array)

#### **Loading Data** and **Modifying Time Column**

In [86]:
powerFailureDf = pd.read_csv('Power Anomalies Simulations Data/AD3_data/01_Power_failure_10sec.csv', sep=';')
powerFailureDf = modifyTimestamps(df=powerFailureDf)

#### **Getting 10 cycles of AC signals** and **Converting to timestamps**

In [87]:
start   =   5555
end     =   start + 4000

V1  =   powerFailureDf['U_L1real'].iloc[start:end].to_numpy()
I1  =   powerFailureDf['I_L1real'].iloc[start:end].to_numpy()

T   =   powerFailureDf['Time'].iloc[start:end].to_numpy()
T   =   LDAP_to_time(t_array=T)

In [88]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            # x = time.to_numpy(),
            y = V1,
            line =  dict(shape =  'spline', color = 'lightskyblue'),
            name = 'Noisy V1',
            ))

fig.add_trace(go.Scatter(
            y = I1,
            line =  dict(shape =  'spline', color = 'lightsalmon'),
            name = 'Noisy I1'
            ))

fig.show()

#### **Defining Low Pass Filter** and **Filtering V and I signals**

In [89]:
def butter_lowpass_filter(signal, cutoff, fs, order):

    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq

    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, signal)
    
    return y

In [90]:
sampling_freq   =   20000.0         # sampling frequency of the signal (Hz)
cutoff_freq     =   100             # desired cutoff frequency of the filter (Hz)
order           =   2               # sine wave can be approx represented as quadratic

V1_filt = butter_lowpass_filter(signal=V1, cutoff=cutoff_freq, fs=sampling_freq, order=order)
I1_filt = butter_lowpass_filter(signal=I1, cutoff=cutoff_freq, fs=sampling_freq, order=order)

In [91]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            # x = time.to_numpy(),
            y = V1,
            line =  dict(shape =  'spline', color = 'lightskyblue'),
            name = 'Noisy U1',
            ))

fig.add_trace(go.Scatter(
            y = V1_filt,
            line =  dict(shape =  'spline', color = 'darkslategrey'),
            name = 'Filtered U1'
            ))

fig.show()

In [92]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = I1,
            line =  dict(shape =  'spline', color = 'lightsalmon'),
            name = 'Noisy I1',
            ))

fig.add_trace(go.Scatter(
            y = I1_filt,
            line =  dict(shape =  'spline', color = 'darkslategrey'),
            name = 'Filtered I1',
            ))

fig.show()

In [93]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = V1_filt,
            line =  dict(shape =  'spline', color = 'darkcyan'),
            name = 'Filtered V1',
            ))

fig.add_trace(go.Scatter(
            y = I1_filt,
            line =  dict(shape =  'spline', color = 'darksalmon'),
            name = 'Filtered I1',
            ))

fig.show()

#### **Calculating Phase Angle**

In [94]:
def phaseAngle(V_array, I_array, time_array):
    
    zero_crossings_V_filt = list(np.asarray((-0.0025 <= V_array) & (V_array <= 0.0025)).nonzero())[0]
    zero_crossings_I_filt = list(np.asarray((-0.0025 <= I_array) & (I_array <= 0.0025)).nonzero())[0]

    # Get the time stamps when V and I cross zero
    time_stamps_V = time_array[zero_crossings_V_filt]
    time_stamps_I = time_array[zero_crossings_I_filt]

    # Compute the phase angle between V and I signals
    td = pd.Timedelta(seconds=1)
    time_delta = (time_stamps_V[0] - time_stamps_I[0]) / td

    phase_angle_degrees = 360 * 50 * time_delta

    return phase_angle_degrees

In [95]:
phase_angle_degrees = phaseAngle(V_array=V1_filt, I_array=I1_filt, time_array=T)

print(f'Phase Angle: {phase_angle_degrees} degrees')

Phase Angle: -2.7180000000000004 degrees


#### **Calculating Power**

In [96]:
def power(V_array, I_array, phase_angle):

    power_array = np.multiply(V_array, I_array) * np.cos(phase_angle)
    return power_array

In [97]:
P1 = power(V_array=V1_filt, I_array=I1_filt, phase_angle=phase_angle_degrees)
P1

array([-0.01976407, -0.01968336, -0.0195972 , ..., -0.01525814,
       -0.01527275, -0.01528469])

In [98]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = P1,
            line =  dict(shape =  'spline', color = 'darkcyan'),
            name = 'P1',
            ))

fig.show()

#### **Calculating FFT**, **Harmonics present in the signal** and **THD**

In [99]:
def THD(abs_data):
    sq_sum = np.sum(np.square(abs_data))

    sq_prime = np.square(np.max(abs_data))
    sq_harmonics = sq_sum - sq_prime

    thd = 100 * np.sqrt(sq_harmonics / sq_prime)

    return thd


def FFT(signal, fs=20000):

    X = fft(signal)
    N = len(X)
    n = np.arange(N)
    T = N/fs

    freq = (n/T)[1:int(N/2)] 
    freq_amplitudes = np.abs(X)[1:int(N/2)]

    sorted_indices = np.argsort(-freq_amplitudes)

    dominant_freq = freq[sorted_indices[0]]
    other_freq = freq[sorted_indices[1:]]

    THD_signal = THD(freq_amplitudes)

    return (freq, freq_amplitudes, dominant_freq, other_freq, THD_signal)

In [103]:
freq, freq_amplitudes, dominant_freq, other_freq, THD_V1_filt = FFT(signal=V1_filt, fs=sampling_freq)

print(f'Prime Harmonic:                     {dominant_freq} Hz')
print(f'Next 10 Harmonics:                  {other_freq[:10]}\n')
print(f'Total Harmonic Distortion (THD):    {THD_V1_filt} %')

fig = go.Figure()

fig.add_trace(go.Scatter(x=freq, y=freq_amplitudes, mode='lines', line=dict(color='blue'), name='FFT Amplitude |X(freq)|'))
fig.update_layout(
    title='FFT Visualization',
    xaxis_title='Freq (Hz)',
    yaxis_title='FFT Amplitude |X(freq)|',
    xaxis=dict(range=[0, sampling_freq/2])
)

fig.show()

Prime Harmonic:                     50.0 Hz
Next 10 Harmonics:                  [ 15. 100.  60. 150.  40.  30.  70.  35.   5.  45.]

Total Harmonic Distortion (THD):    3.029496060970682 %


#### **Calculating RMS Voltage** according to **IEEE 1564-2024**

In [101]:
def rms_cycle(V_array, time_array):

    for i in np.arange(start=0, stop=len(V_array), step=200):
        start = i
        end = i + 400

        rms = np.sqrt(np.mean(np.square(V1_filt[start:end])))

        print(f'Cycle Start: {time_array[i]}  |  RMS: {rms} V')


rms_cycle(V_array=V1_filt, time_array=T)

print(f'\nRMS voltage of complete signal: {np.max(V1_filt)/np.sqrt(2)} V')

Cycle Start: 2024-02-23 11:42:46.784750  |  RMS: 0.1097101762478802 V
Cycle Start: 2024-02-23 11:42:46.794750  |  RMS: 0.11178867842578734 V
Cycle Start: 2024-02-23 11:42:46.804750  |  RMS: 0.11307452296846096 V
Cycle Start: 2024-02-23 11:42:46.814751  |  RMS: 0.11383499606573626 V
Cycle Start: 2024-02-23 11:42:46.824751  |  RMS: 0.11326717608694209 V
Cycle Start: 2024-02-23 11:42:46.834751  |  RMS: 0.11317292416854668 V
Cycle Start: 2024-02-23 11:42:46.844749  |  RMS: 0.11321781371968802 V
Cycle Start: 2024-02-23 11:42:46.854750  |  RMS: 0.11291067198087434 V
Cycle Start: 2024-02-23 11:42:46.864750  |  RMS: 0.11207205763013695 V
Cycle Start: 2024-02-23 11:42:46.874750  |  RMS: 0.11124152767431791 V
Cycle Start: 2024-02-23 11:42:46.884750  |  RMS: 0.11165494877789668 V
Cycle Start: 2024-02-23 11:42:46.894751  |  RMS: 0.11361027145461088 V
Cycle Start: 2024-02-23 11:42:46.904751  |  RMS: 0.1144068252430332 V
Cycle Start: 2024-02-23 11:42:46.914751  |  RMS: 0.11397862422885095 V
Cycle St